In [24]:
from math import sqrt
from itertools import product

import pandas as pd
import torch
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from IPython.display import Image

# VGG module
- なぜ34層なのか？　(まだわかっていない)
- 本が間違っている : 35層ある

![VGG層](../data/vgg.png)

## 実装の詳細
- M : 床関数モード floor
- MC : 天井関数モード ceil
- conv2d
    - dilation : a trous algorithmを可能にする
        - simulation : https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md
    - kernel_size : フィルタサイズ

In [25]:
def vgg():
    layers = []
    in_channels = 3
    
    layer_conf = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'MC',
                 512,512,512,'M', 512,512,512]
    
    for layer in layer_conf:
        if layer == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif layer == 'MC':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            out_channels = layer
            conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = out_channels
    
    # 5番目のプールは違う設定になる
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    # 上ですでに13層のconvを定義済み
    conv14 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv15 = nn.Conv2d(1024, 1024, kernel_size=1)
    
    layers += [pool5, conv14, nn.ReLU(inplace=True), conv15, nn.ReLU(inplace=True)]
    return nn.ModuleList(layers)

In [26]:
vgg_test = vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1

# extrasの実装
- ReLU関数は順伝播関数の中で用意することにする。何で？

In [27]:
def extras():
    layers = []
    in_channels = 1024
   
    #  extrasの設定が間違っている
#     layer_conf = [512, 512, 256, 256, 256, 256, 512, 512]
    layer_conf = [512, 512, 256, 256, 256, 256, 256, 256]

    layers += [nn.Conv2d(in_channels, layer_conf[0], kernel_size=(1))]
    layers += [nn.Conv2d(layer_conf[0], layer_conf[1], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(layer_conf[1], layer_conf[2], kernel_size=(1))]
    layers += [nn.Conv2d(layer_conf[2], layer_conf[3], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(layer_conf[3], layer_conf[4], kernel_size=(1))]
    layers += [nn.Conv2d(layer_conf[4], layer_conf[5], kernel_size=(3))]
    layers += [nn.Conv2d(layer_conf[5], layer_conf[6], kernel_size=(1))]
    layers += [nn.Conv2d(layer_conf[6], layer_conf[7], kernel_size=(3))]
    
    # 活性化関数のReLUは今回はSSDモデルの順伝搬のなかで用意することにし、
    # extraモジュールでは用意していません

    return nn.ModuleList(layers)

In [28]:
extras_test = extras()
print(extras_test)

ModuleList(
  (0): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
)


# LocationとConfidence moduleの実装
- bbox_aspect_num で各inputで使うbounding boxの数を渡す
- 本では、この二つのmoduleを一緒に定義してあるが、わかり安くするため分離する
- input_channels: locationとconfidence layerの入力となるデータのchannel数

In [29]:
def location_layers(input_channels=[512, 1024, 512, 256, 256, 256],
                    bbox_aspect_num=[4,6,6,6,4,4]):
    location_layers = []
    
    # locationの座標が4次元のリストになるので、*4になる
    for idx, input_channel in enumerate(input_channels):
        location_layers += [
            nn.Conv2d(input_channel, bbox_aspect_num[idx]*4, kernel_size=3, padding=1)
        ]
    return nn.ModuleList(location_layers)

In [30]:
def confidence_layers(num_classes=21,
                      input_channels=[512, 1024, 512, 256, 256, 256],
                      bbox_aspect_num=[4,6,6,6,4,4]
                     ):
    confidence_layers = []
    for idx, input_channel in enumerate(input_channels):
        confidence_layers += [
            nn.Conv2d(input_channel, bbox_aspect_num[idx]*num_classes, kernel_size=3, padding=1)
        ]
    return nn.ModuleList(confidence_layers)    

In [31]:
loc_layers = location_layers()
conf_layers = confidence_layers()

In [32]:
loc_layers

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [33]:
conf_layers

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

# L2Norm層の実装
- 今回のL2Normは 次元(512, 38, 38)のsource1のtensorに対して行う
- 38x38個の要素ごとに、512channleにわたる二乗和のルートを計算し、その値を512x38x38の全ての値にそれぞれ割ることで正規化を行う。(説明難しい！)
- 言うまでもなくこの処理で各特徴量の値の範囲が一定範囲に収まることになる
- チャンネルごとに学習必要な係数をかけることで重み付けを行う。なぜ？
- unsqueezeがよくわかっていない？

In [34]:
class L2Norm(nn.Module):
    def __init__(self, input_channels=512, scale=20):
        super(L2Norm, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(input_channels))
        self.scale = scale
        self.init_parameters()
        self.eps = 1e-10

    def init_parameters(self):
        # weightの初期値が全てscaleになる
        init.constant_(self.weight, self.scale)  
        
    def forward(self, x):
        # normのtensorサイズは [bach_num, 1, 38, 38]になる
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt() + self.eps
        x = torch.div(x, norm)
        
        # weightsのサイズが [batch_num, 512, 38, 38]になる
        weights = self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)
        print(weights.shape)
        out = weights * x
        
        return out

# デフォルトボックスの実装
- default boxの種類
    - 四つの場合: 大きい正方形、小さい正方形, 縦 : 横 = 1 : 2, 縦 : 横 = 2 : 1
    - 六つの場合: 縦 : 横 = 3 : 1, 縦 : 横 = 1 : 3 のボックスを上に追加
- デフォルトボックスサイズの設定が本と違う気がする
    - 実際には 3:1, 2:1とかになっていない


In [35]:
class DefaultBox(object):
    def __init__(self, config):
        super(DefaultBox, self).__init__()
        
        # 初期設定
        self.image_size = config['input_size']
        
        # location, confidence layerへinputするdataサイズ
        self.input_feature_maps = config['input_feature_maps']
        self.input_len = len(config['input_feature_maps'])
        self.dbox_pixel_sizes = config['dbox_pixel_sizes']
        self.small_dbox_sizes = config['small_dbox_sizes']
        self.big_dbox_sizes = config['big_dbox_sizes']
        self.dbox_aspect_ratios = config['dbox_aspect_ratios']

    def create_dbox_list(self):
        mean = []
        
        for idx, feature_map_size in enumerate(self.input_feature_maps):
            for feature_map_x, feature_map_y in product(range(feature_map_size), repeat=2):
                
                # feature_mapを適応後の画像サイズ
                feature_image_size = self.image_size / self.dbox_pixel_sizes[idx]
                
                # default boxの中心座標
                center_x = (feature_map_x+0.5) / feature_image_size
                center_y = (feature_map_y+0.5) / feature_image_size
                
                # 小さいdefault box : 中心は変わらない
                small_dbox = self.small_dbox_sizes[idx] /self.image_size
                mean += [center_x, center_y, small_dbox, small_dbox]
                
                # 大きいdefault box : 中心は変わらない
                big_dbox = sqrt(small_dbox*(self.big_dbox_sizes[idx]/self.image_size))
                mean += [center_x, center_y, big_dbox, big_dbox]
                
                # 長方形default box
                for dar in self.dbox_aspect_ratios[idx]:
                    mean += [center_x, center_y, small_dbox*sqrt(dar), small_dbox/sqrt(dar)]
                    mean += [center_x, center_y, small_dbox/sqrt(dar), small_dbox*sqrt(dar)]
        
        print('mean: ' ,len(mean))
        # default boxを torch.size([8732, 4])に変換
        output = torch.Tensor(mean).view(-1, 4)
        
        # DBoxが画像の外にはみ出るのを防ぐため、大きさを最小0、最大1にする
        # 1以上の値は全て1に、0以下の値は全てを0に設定する気がする
        output.clamp_(max=1, min=0)
        
        return output

# DBox作成

In [36]:
ssd_config = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'input_feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'dbox_pixel_sizes': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'small_dbox_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'big_dbox_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'dbox_aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

dbox = DefaultBox(ssd_config)
dbox_list = dbox.create_dbox_list()


print(dbox_list)


# DBoxの出力を確認する
pd.DataFrame(dbox_list.numpy())

mean:  34928
tensor([[0.0133, 0.0133, 0.1000, 0.1000],
        [0.0133, 0.0133, 0.1414, 0.1414],
        [0.0133, 0.0133, 0.1414, 0.0707],
        ...,
        [0.5000, 0.5000, 0.9612, 0.9612],
        [0.5000, 0.5000, 1.0000, 0.6223],
        [0.5000, 0.5000, 0.6223, 1.0000]])


,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.013333,0.040000,0.100000,0.100000
5,0.013333,0.040000,0.141421,0.141421
6,0.013333,0.040000,0.141421,0.070711
7,0.013333,0.040000,0.070711,0.141421
8,0.013333,0.066667,0.100000,0.100000
9,0.013333,0.066667,0.141421,0.141421


# SSDクラスの実装

In [37]:
class SSD(nn.Module):
    def __init__(self, phase, config):
        super(SSD, self).__init__()
        
        self.phase = phase  # train or inference
        self.num_classes = config['num_classes']
        
        # SSDの組み立て
        self.vgg = vgg()
        self.extras = extras()
        self.L2Norm = L2Norm()
        
        # num_classes=21,
        # input_channels=[512, 1024, 512, 256, 256, 256],
        # bbox_aspect_num=[4,6,6,6,4,4]
        self.location = location_layers(config['input_channels'],
                                        config['bbox_aspect_num'])
        self.confidence = confidence_layers(config['num_classes'],
                                            config['input_channels'],
                                            config['bbox_aspect_num'])
        
        # Default Boxの作成
        dbox = DefaultBox(config)
        self.dbox_list = dbox.create_dbox_list()
        
        if phase == 'inference':
            self.detect = Detect()

In [38]:
ssd_config = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'input_feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'dbox_pixel_sizes': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'small_dbox_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'big_dbox_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'dbox_aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'input_channels': [512, 1024, 512, 256, 256, 256],
}

ssd_test =  SSD(phase="train", config=ssd_config)
print(ssd_test)

mean:  34928
SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (17):

# Forward Propagationの実装
- Non Maximum Suppression
- ![座標変換](../data/convert_position.png)

In [39]:
def decode(loc, dbox_list):
    # オフセット情報からBBoxを求める
    boxes = torch.cat((
        dbox_list[:, :2] + loc[:, :2] * 0.1 * dbox_list[:, 2:],
        dbox_list[:, 2:] * torch.exp(loc[:, 2:] * 0.2)), dim=1)
    # boxesのサイズはtorch.Size([8732, 4])となります

    # BBoxの座標情報を[cx, cy, width, height]から[xmin, ymin, xmax, ymax] に
    boxes[:, :2] -= boxes[:, 2:] / 2  # 座標(xmin,ymin)へ変換
    boxes[:, 2:] += boxes[:, :2]  # 座標(xmax,ymax)へ変換

    return boxes

# Non-Maximum Suppressionの実装
- 説明文書
    - http://meideru.com/archives/3538
- 同じ物体に対して、複数のBouding Boxが検知した時に、一つのBouding Boxにまとめる処理
- Bounding Box間で一定閾値以上の面積が被っている場合、それらは同じ物体に対する予測結果だと皆し、Non-Maximum Suppressionの処理対象になる
- 最終的に唯一残るのは、confidence値が一番高いBouding Boxである
- ここはやや複雑で、記事で詳しく説明すると良いかも

In [44]:
def nm_suppression(boxes, scores, overlap=0.45, top_k=200):
    """
    1. 高速化のため、一定confidenceを持つbouding boxのみが処理対処になる
    2. overlap : 複数のbouding boxの被る面積の閾値
    
    :param:
    boxes : confidence一定以上のbounding box
    scores: boxesで指定したbounding boxらのconfidence値
    
    :return:
    keep: 生き残ったbounding boxのindex情報, confidenceの降順
    count: nmsで生き残ったbounding box数
    """
    count = 0
    # torch.Size([入力されたbouding box数])
    keep = scores.new(scores.size(0)).zero_().long()
    
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = torch.mul(x2-x1, y2-y1)
    
    # 後で使う?
    tmp_x1 = boxes.new()
    tmp_y1 = boxes.new()
    tmp_x2 = boxes.new()
    tmp_y2 = boxes.new()
    tmp_w = boxes.new()
    tmp_h = boxes.new()

    #  scoreを昇順にソーティング
    v, idx = scores.sort(0)
    
    # top_k 個を選ぶ理由は？
    idx = idx[-top_k:]
    
    while idx.numel() > 0:
        i = idx[-1]

        # keepの現在の最後にconf最大のindexを格納する
        # このindexのBBoxと被りが大きいBBoxをこれから消去する
        keep[count] = i
        count += 1

        # 最後のBBoxになった場合は、ループを抜ける
        if idx.size(0) == 1:
            break

        # 現在のconf最大のindexをkeepに格納したので、idxをひとつ減らす
        idx = idx[:-1]

        # -------------------
        # これからkeepに格納したBBoxと被りの大きいBBoxを抽出して除去する
        # -------------------
        # ひとつ減らしたidxまでのBBoxを、outに指定した変数として作成する
        torch.index_select(x1, 0, idx, out=tmp_x1)
        torch.index_select(y1, 0, idx, out=tmp_y1)
        torch.index_select(x2, 0, idx, out=tmp_x2)
        torch.index_select(y2, 0, idx, out=tmp_y2)

        # すべてのBBoxに対して、現在のBBox=indexがiと被っている値までに設定(clamp)
        tmp_x1 = torch.clamp(tmp_x1, min=x1[i])
        tmp_y1 = torch.clamp(tmp_y1, min=y1[i])
        tmp_x2 = torch.clamp(tmp_x2, max=x2[i])
        tmp_y2 = torch.clamp(tmp_y2, max=y2[i])

        # wとhのテンソルサイズをindexを1つ減らしたものにする
        tmp_w.resize_as_(tmp_x2)
        tmp_h.resize_as_(tmp_y2)

        # clampした状態でのBBoxの幅と高さを求める
        tmp_w = tmp_x2 - tmp_x1
        tmp_h = tmp_y2 - tmp_y1

        # 幅や高さが負になっているものは0にする
        tmp_w = torch.clamp(tmp_w, min=0.0)
        tmp_h = torch.clamp(tmp_h, min=0.0)

        # clampされた状態での面積を求める
        inter = tmp_w*tmp_h
    
        # Intersection over Union
        # IoU = intersect部分 / (area(a) + area(b) - intersect部分)の計算
        rem_areas = torch.index_select(area, 0, idx)  # 各BBoxの元の面積
        union = (rem_areas - inter) + area[i]  # 2つのエリアのANDの面積
        IoU = inter/union

        # IoUがoverlapより小さいidxのみを残す
        idx = idx[IoU.le(overlap)]  # leはLess than or Equal toの処理をする演算です
        # IoUがoverlapより大きいidxは、最初に選んでkeepに格納したidxと同じ物体に対してBBoxを囲んでいるため消去

    # whileのループが抜けたら終了

    return keep, count

# Detect クラスの実装
- Bounding Boxの情報を得る [x_min, y_min, x_max, y_max]
- confidenceが一定以上のBounding Boxを取り出す
- Non-Maximum Suppressionを行い、最終的な検知Bounding Boxを残す

In [41]:
class Detect(Function):
    def __init__(self, confidence_thresh=0.01, top_k=200, nms_thresh=0.45):
        # confidenceの値を正規化する時に使う
        self.softmax = nn.Softmax(dim=-1)
        self.confidence_thresh = confidence_thresh
        self.top_k = top_k
        
        # 　Bounding Boxで被る面積がこの閾値以上の場合、Non-Maximum Suppression
        #  の処理対象と考える
        self.nms_thresh = nms_thresh

    def forward(self, location_data, confidence_data, dbox_list):
        """
        順伝播の計算を実行する
        
        :return:
        output : torch.Size([batch_num, 21, 200, 5])
        (batch_num, クラス, top200のconfidence, Bouding Boxの情報)
        """
        
        num_batch = location_data.size(0)
        # Default Boxの数 : 8732
        num_dbox = location_data.size(1)
        # 予測クラスの数 : 21
        num_classes = confidence_data.size(2)
        
        confidence_data = self.softmax(confidence_data)
        
        # なぜ5？
        output = torch.zeros(num_batch, num_classes, self.top_k, 5)
        
        # confidence_dataを[batch_num,8732,num_classes]から[batch_num, num_classes,8732]に順番変更
        confidence_detections = confidence_data.transpose(2, 1)
        
        for i in range(num_batch):
            # Bounding Boxの位置情報の計算
            decoded_boxes = decode(location_data[i], dbox_list)
            confidence_scores = confidence_detections[i].clone()

            for cl in range(1, num_classes):
                # 0は背景クラスになるため無視
                
                # 閾値を超えたものを1に、以下のものを0にする処理
                #  この処理の結果を使って、閾値を超えたBouding Boxのフィルタリングを行う
                # confidence_scores:torch.Size([21, 8732])
                # c_mask:torch.Size([8732])
                c_mask = confidence_scores[cl].gt(self.confidence_thresh)

                # scoresはtorch.Size([閾値を超えたBBox数])
                scores = conf_scores[cl][c_mask]

                # 閾値を超えたconfがない場合、つまりscores=[]のときは、何もしない
                if scores.nelement() == 0:  # nelementで要素数の合計を求める
                    continue

                # c_maskを、decoded_boxesに適用できるようにサイズを変更します
                l_mask = c_mask.unsqueeze(1).expand_as(decoded_boxes)
                # l_mask:torch.Size([8732, 4])

                # l_maskをdecoded_boxesに適応します
                boxes = decoded_boxes[l_mask].view(-1, 4)
                # decoded_boxes[l_mask]で1次元になってしまうので、
                # viewで（閾値を超えたBBox数, 4）サイズに変形しなおす

                # Non-Maximum Suppressionを実施
                ids, count = nm_suppression(
                    boxes, scores, self.nms_thresh, self.top_k)
                # ids：confの降順にNon-Maximum Suppressionを通過したindexが格納
                # count：Non-Maximum Suppressionを通過したBBoxの数

                # outputにNon-Maximum Suppressionを抜けた結果を格納
                output[i, cl, :count] = torch.cat((scores[ids[:count]].unsqueeze(1),
                                                   boxes[ids[:count]]), 1)

        return output  # torch.Size([1, 21, 200, 5])
        

# SSDクラスの実装

In [42]:
class SSD(nn.Module):
    def __init__(self, phase, config):
        super(SSD, self).__init__()
        
        self.phase = phase  # train or inference
        self.num_classes = config['num_classes']
        
        # SSDの組み立て
        self.vgg = vgg()
        self.extras = extras()
        self.L2Norm = L2Norm()
        
        # num_classes=21,
        # input_channels=[512, 1024, 512, 256, 256, 256],
        # bbox_aspect_num=[4,6,6,6,4,4]
        self.location = location_layers(config['input_channels'],
                                        config['bbox_aspect_num'])
        self.confidence = confidence_layers(config['num_classes'],
                                            config['input_channels'],
                                            config['bbox_aspect_num'])
        
        # Default Boxの作成
        dbox = DefaultBox(config)
        self.dbox_list = dbox.create_dbox_list()
        
        if phase == 'inference':
            self.detect = Detect()

    def forward(self, x):
        inputs = list()  # ６種のinputを格納する
        location = list()
        confidence = list()
        
        for k in range(23):
            x = self.vgg[k](x)
        
        input1 = self.L2Norm(x)
        inputs.append(input1)
        
        for k in range(23, len(self.vgg)):
            x = self.vgg[k][x]
        inputs.append(x)
        
        # input 3 ~ 6 までを抽出
        for k in range(self.extras):
            x = F.relu(v(x), inplace=True)
            if k%2 == 1:
                inputs.append(x)

        # 各inputに対して、 locationとconfidenceをそれぞれ計算
        # [batch_num, feature_map数, feature_map数, 4*aspect_ratio種類数]
        for (x, l, c) in zip(inputs, self.location, self.confidence):
            location.append(l(x).permute(0,2,3,1).contiguous())
            confidence.append(c(x).permute(0,2,3,1).contiguous())
            
        # locationのサイズは、torch.Size([batch_num, 34928])
        # confidenceのサイズはtorch.Size([batch_num, 34928*num_classes])になる
        location = torch.cat([o.view(o.size(0), -1) for o in location], 1)
        confidence = torch.cat([o.view(o.size(0), -1) for o in confidence], 1)
        
        # locationのサイズは、torch.Size([batch_num, 8732, 4])
        # confidenceのサイズは、torch.Size([batch_num, 8732, 21])
        location = location.view(location.size(0), -1, 4)
        confidence = confidence.view(confidence.size(0), -1, self.num_classes)
        
        output = (location, confidence, self.dbox_list)
        
        if self.phase == 'inference':
            return self.detect(output[0], output[1], output[2])
        else:
            return output